
# ReAct with LangGraph + OpenAI — End‑to‑End Tutorial

This notebook implements the **ReAct** loop (Reason → Act → Observe → Repeat → Answer) using **LangGraph** for control flow and the **OpenAI API** for the language model with **tool calling**.

### What you'll learn
- How to encode the ReAct loop using **LangGraph**'s `StateGraph`
- How to let an OpenAI model call **tools** (calculator + toy search) via function‑calling
- How to build a **tool-execution node** and loop model→tools→model until a **final answer**
- How to run **worked examples** (math, retrieval, multi‑hop)



> ⚙️ **Requirements**
>
> - Python ≥ 3.9  
> - Packages: `openai`, `langgraph`, `pydantic` (install below)  
> - An environment variable `OPENAI_API_KEY` set with a valid key


In [31]:

# If needed, install the libraries (uncomment and run)
# %pip install -U openai langgraph pydantic



## 1) ReAct + LangGraph: how pieces fit

- **Model Node**: calls the OpenAI chat model with tool schemas.  
  The model can output:
  - a normal message (finish, respond to user), or
  - one or more **tool calls** (structured function calls).

- **Tools Node**: executes each requested tool, returns **observations** as messages
  back to the model.

- **Control Flow**: LangGraph loops: `Model → (has tool calls?) → Tools → Model ... → Final Answer`.


In [32]:

from __future__ import annotations

import os, json, math, ast, operator, re
from typing import TypedDict, List, Dict, Any

from openai import OpenAI
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, START, END
from langgraph.types import Command


In [33]:

# Create OpenAI client
# Make sure your environment has:  export OPENAI_API_KEY=sk-...
#client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
MODEL_NAME = "gpt-4o-mini"  # choose a tool-capable model



## 2) Define Tools

We provide two simple tools:
1. **calculator(query: str)** — safe arithmetic using a constrained AST evaluator
2. **search(query: str)** — keyword search over a tiny in‑memory KB

We'll expose them to the OpenAI model via **function (tool) schemas** so it can call them.


In [34]:

# --- Calculator ---
def _safe_eval(expr: str) -> str:
    node = ast.parse(expr, mode="eval")
    allowed_ops = {
        ast.Add: operator.add, ast.Sub: operator.sub,
        ast.Mult: operator.mul, ast.Div: operator.truediv,
        ast.Pow: operator.pow, ast.Mod: operator.mod,
        ast.FloorDiv: operator.floordiv, ast.USub: operator.neg
    }
    def eval_node(n):
        if isinstance(n, ast.Expression):
            return eval_node(n.body)
        if hasattr(ast, "Constant") and isinstance(n, ast.Constant):
            if isinstance(n.value, (int, float)):
                return n.value
            raise ValueError("Only numeric constants allowed.")
        if isinstance(n, ast.Num):
            return n.n
        if isinstance(n, ast.BinOp):
            op = allowed_ops.get(type(n.op))
            if not op: raise ValueError("Operator not allowed.")
            return op(eval_node(n.left), eval_node(n.right))
        if isinstance(n, ast.UnaryOp) and type(n.op) in (ast.USub,):
            return allowed_ops[type(n.op)](eval_node(n.operand))
        raise ValueError("Expression not allowed.")
    return str(eval_node(node))

def calculator(query: str) -> str:
    try:
        # normalize `x` to `*`, keep only math tokens
        expr = re.sub(r'[^0-9\+\-\*/\.\(\)\s]', '', query.replace('x','*'))
        return _safe_eval(expr)
    except Exception as e:
        return f"ERROR: {e}"


In [35]:

# --- Tiny Search KB ---
KB = {
    "capital_of_canada": "Ottawa is the capital of Canada.",
    "ottawa_population": "Ottawa metro population is ~1,000,000 (approx.).",
    "react_definition": "ReAct combines reasoning traces with tool use in a loop (Yao et al., 2022)."
}

def search(query: str) -> str:
    q = query.lower().strip()
    if not q:
        return "ERROR: empty query"
    hits = []
    for k, v in KB.items():
        text = (k + " " + v).lower()
        if all(tok in text for tok in q.split()):
            hits.append(f"- {k}: {v}")
    return "Results:\n" + "\n".join(hits) if hits else "No results."



## 3) Tool Schemas (function-calling)

We describe each tool (name, parameters) so the model can call it when helpful.


In [36]:

# OpenAI "tools" parameter expects a specific JSON schema for functions
TOOLS_SCHEMA = [
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Evaluate basic arithmetic expressions, e.g., '23 * 47'.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The math expression to evaluate."}
                },
                "required": ["query"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "Keyword search on a tiny in-memory knowledge base.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Search string."}
                },
                "required": ["query"]
            },
        },
    },
]



## 4) Graph State and utilities

We keep a **chat transcript** as our state. The model may return **tool calls**; then we execute tools and append **observations** back as messages.


In [37]:

class AgentState(TypedDict):
    messages: List[Dict[str, Any]]  # OpenAI-style messages: {role, content, tool_call_id?, name?}


In [38]:

def has_tool_calls(openai_message: Dict[str, Any]) -> bool:
    tool_calls = openai_message.get("tool_calls") or []
    return len(tool_calls) > 0



## 5) Node: Model call

This node sends the running transcript to the OpenAI model with `tools=TOOLS_SCHEMA`.  
The model either replies with a **final message** or with one or more **tool calls**.


In [39]:
def call_model(state: AgentState) -> AgentState:
    messages = state.get("messages", [])
    if not messages or messages[0].get("role") != "system":
        messages = [{"role": "system", "content":
            "You are a helpful ReAct agent. Call tools when needed; keep chain-of-thought implicit; be concise."
        }] + messages

    resp = client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        tools=TOOLS_SCHEMA,
        tool_choice="auto",
        temperature=0.2,
    )
    m = resp.choices[0].message

    # Append as a minimal dict (role/content/tool_calls) to keep a consistent shape
    messages.append({
        "role": "assistant",
        "content": m.content,
        "tool_calls": [
            {
                "id": tc.id,
                "type": "function",
                "function": {
                    "name": tc.function.name,
                    "arguments": tc.function.arguments,
                },
            } for tc in (m.tool_calls or [])
        ],
    })
    return {"messages": messages}



## 6) Node: Tool execution

- Read pending tool calls from the last model message
- Execute each tool in Python
- Append **tool results** as messages of role `"tool"` with matching `tool_call_id`


In [40]:

def call_tools(state: AgentState) -> AgentState:
    messages = state.get("messages", [])
    if not messages:
        return {"messages": messages}
    last = messages[-1]

    tool_calls = last.get("tool_calls") or []
    for tc in tool_calls:
        fn_name = tc.function.name
        args = {}
        try:
            args = json.loads(tc.function.arguments or "{}")
        except Exception:
            pass

        # Dispatch to our Python tools
        if fn_name == "calculator":
            result = calculator(**args)
        elif fn_name == "search":
            result = search(**args)
        else:
            result = f"ERROR: Unknown tool '{fn_name}'"

        # Add the observation so the model can read it
        messages.append({
            "role": "tool",
            "tool_call_id": tc.id,
            "name": fn_name,
            "content": result,
        })
    return {"messages": messages}



## 7) Build the LangGraph

- `START → model`
- If the model returns **tool calls**, go to `tools` then back to `model`
- Else, stop at `END` (we have a final answer)


In [41]:

# --- Helper to normalize OpenAI tool_call shapes (dict or object) ---
def _tc_fields(tc):
    """
    Returns (tool_call_id, name, arguments_json_str) for either:
    - dict-shaped tool call: {"id": "...", "function": {"name": "...", "arguments": "..."}}
    - object-shaped tool call: tc.id, tc.function.name, tc.function.arguments
    """
    if isinstance(tc, dict):
        fn = tc.get("function") or {}
        return tc.get("id"), fn.get("name"), fn.get("arguments")
    # object-like (SDK types)
    return getattr(tc, "id", None), getattr(tc.function, "name", None), getattr(tc.function, "arguments", None)

def call_tools(state: AgentState) -> AgentState:
    messages = state.get("messages", [])
    if not messages:
        return {"messages": messages}

    last = messages[-1]
    tool_calls = last.get("tool_calls") or []

    for tc in tool_calls:
        tc_id, fn_name, args_json = _tc_fields(tc)
        try:
            args = json.loads(args_json or "{}")
        except Exception:
            args = {}

        # Dispatch to our Python tools
        if fn_name == "calculator":
            result = calculator(**args)
        elif fn_name == "search":
            result = search(**args)
        else:
            result = f"ERROR: Unknown tool '{fn_name}'"

        # Append observation for the model to read
        messages.append({
            "role": "tool",
            "tool_call_id": tc_id,
            "name": fn_name,
            "content": result,
        })

    return {"messages": messages}


def should_continue(state: AgentState) -> str:
    # If last assistant message contains tool calls, run tools; else, we are done.
    messages = state.get("messages", [])
    if not messages:
        return "end"
    last = messages[-1]
    if last.get("role") == "assistant" and has_tool_calls(last):
        return "tools"
    return "end"

graph = StateGraph(AgentState)
graph.add_node("model", call_model)
graph.add_node("tools", call_tools)

graph.add_edge(START, "model")
graph.add_conditional_edges(
    "model",
    should_continue,
    {
        "tools": "tools",
        "end": END
    }
)
graph.add_edge("tools", "model")

app = graph.compile()



## 8) Helper runner

`run_react(query)` seeds the chat with the user message, runs the graph, and prints the transcript.


In [46]:

def run_react(query: str, max_steps: int = 8) -> List[Dict[str, Any]]:
    state: AgentState = {
        "messages": [
            {"role": "user", "content": query}
        ]
    }
    # Each `app.invoke` will advance until END, but we rely on internal loop.
    # `max_steps` serves as a guard if the model/tool loop fails to converge.
    for _ in range(max_steps):
        state = app.invoke(state)
        # After invoke, if the last message is from assistant with no tool_calls, we're done
        last = state["messages"][-1]
        if last["role"] == "assistant" and not has_tool_calls(last):
            break
    return state["messages"]

def print_transcript(messages: List[Dict[str, Any]]):
    for m in messages:
        role = m.get("role")
        if role == "assistant":
            print("\n[assistant]")
            if m.get("tool_calls"):
                print("(assistant requested tools)")
        elif role == "tool":
            print(f"\n[tool:{m.get('name')}] → {m.get('content')}")
        elif role == "user":
            print(f"\n[user] {m.get('content')}")
        elif role == "system":
            pass  # omit for brevity



## 9) Examples

> Note: These require a valid `OPENAI_API_KEY` and internet access.


### 9.1 Math (calculator)

In [47]:

msgs = run_react("What is 23 * 47? Please compute it.")
print_transcript(msgs)


/tmp/ipykernel_1782643/1016456093.py:17: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(n, ast.Num):



[user] What is 23 * 47? Please compute it.

[assistant]
(assistant requested tools)

[tool:calculator] → 1081

[assistant]


### 9.2 Retrieval (search)

In [48]:

msgs = run_react("What is ReAct in language models?")
print_transcript(msgs)



[user] What is ReAct in language models?

[assistant]
(assistant requested tools)

[tool:search] → No results.

[assistant]


### 9.3 Multi‑hop (search → compute)

In [49]:

msgs = run_react("What is the population of the capital of Canada?")
print_transcript(msgs)



[user] What is the population of the capital of Canada?

[assistant]
(assistant requested tools)

[tool:search] → No results.

[assistant]
(assistant requested tools)

[tool:search] → No results.

[assistant]



## 10) Extensions, Safety & Tips

- **Guardrails**: validate tool inputs, cap steps with `max_steps`, and block risky code tools.
- **Citations**: require the model to cite which tool results it used before final answers.
- **Caching**: memoize tool outputs to avoid repeated calls.
- **Memory**: store facts learned during the session and surface them to the model.
- **UI**: plug this graph into Streamlit or FastAPI for a simple ReAct assistant.
